<a href="https://colab.research.google.com/github/itsmepriyabrata/priyabrata_ai_python/blob/main/NLP%20part%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Glove(global vectors for word)

In [4]:
pip install gensim


In [5]:
from gensim.utils import simple_preprocess
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from gensim.models.word2vec import Text8Corpus
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors

documents = [
    "The quick brown fox jumps over the lazy dog.",
    "Never jump over the lazy dog quickly.",
    "A lazy dog is quick to jump over."
]

tokenized_documents = [simple_preprocess(doc) for doc in documents]

print("Tokenized Documents:")
print(tokenized_documents)


Tokenized Documents:
[['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog'], ['never', 'jump', 'over', 'the', 'lazy', 'dog', 'quickly'], ['lazy', 'dog', 'is', 'quick', 'to', 'jump', 'over']]


BERT

In [2]:
pip install transformers torch


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [3]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
import pandas as pd

data = {
    'text': [
        'I love programming.',
        'Python is a great language.',
        'I enjoy learning new things.',
        'Debugging is fun.',
        'Coding is exciting.',
        'I hate bugs.',
        'Errors are frustrating.',
        'I dislike poorly written code.',
        'Debugging is challenging.',
        'Writing clean code is satisfying.'
    ],
    'label': [1, 1, 1, 1, 1, 0, 0, 0, 0, 1]
}

df = pd.DataFrame(data)

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(), df['label'].tolist(), test_size=0.2, random_state=42)

class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


MAX_LEN = 32
BATCH_SIZE = 4
EPOCHS = 3
LEARNING_RATE = 1e-5

train_dataset = TextDataset(train_texts, train_labels, tokenizer, MAX_LEN)
val_dataset = TextDataset(val_texts, val_labels, tokenizer, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

# Training loop
for epoch in range(EPOCHS):
    model.train()
    total_loss_train = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss_train += loss.item()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch + 1}, Train loss: {total_loss_train / len(train_loader)}')

    model.eval()
    total_loss_val = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['label']
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss_val += loss.item()
    print(f'Epoch {epoch + 1}, Validation loss: {total_loss_val / len(val_loader)}')

print('Training complete.')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train loss: 0.7042417526245117
Epoch 1, Validation loss: 0.7323291897773743
Epoch 2, Train loss: 0.6144579648971558
Epoch 2, Validation loss: 0.7339383363723755
Epoch 3, Train loss: 0.6022780537605286
Epoch 3, Validation loss: 0.7390624284744263
Training complete.


GPT

In [4]:
pip install transformers torch


In [6]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

def generate_text(prompt, max_length=50, num_return_sequences=1):
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    outputs = model.generate(
        inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        no_repeat_ngram_size=2,
        early_stopping=True
    )

    generated_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return generated_texts

prompt = "Once upon a time"
generated_texts = generate_text(prompt, max_length=100, num_return_sequences=3)

for i, text in enumerate(generated_texts):
    print(f"Generated Text {i+1}:\n{text}\n")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text 1:
Once upon a time I went to the kitchen where I was talking to my grandmother, and I got her out of a suitcase, went in and put it on. She was having a stroke, so I called her and told her what was going on, but we hadn't been to a doctor. When I told the doctor that there was no need to have a catheter, he laughed, told me I should just call the ambulance, called the police on me, I didn't know where this

Generated Text 2:
Once upon a time, when the last was the most valuable, then our economy was able to create new economic, social and industrial goods, and these goods and services were being exported. And all of our business was good. The only thing that has changed is our production. It was necessary for us to produce and provide goods.

Today's economy is different: it doesn't give more to the rich, but rather gives away more of that wealth to poor nations. In the United States,

Generated Text 3:
Once upon a time, she seemed to become a kind of guardian angel, p

# Seq2Seq models

In [7]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
import pandas as pd

# Sample dataset
data = {
    'text': [
        'The quick brown fox jumps over the lazy dog.',
        'Artificial intelligence is transforming the world.',
        'Natural language processing is a subfield of AI.',
        'Seq2Seq models are used for various NLP tasks.',
        'Machine learning algorithms improve with more data.'
    ],
    'summary': [
        'Fox jumps over dog.',
        'AI is changing the world.',
        'NLP is a subfield of AI.',
        'Seq2Seq models for NLP tasks.',
        'ML algorithms improve with data.'
    ]
}

df = pd.DataFrame(data)

train_texts, val_texts, train_summaries, val_summaries = train_test_split(
    df['text'].tolist(), df['summary'].tolist(), test_size=0.2, random_state=42)

class TextSummaryDataset(Dataset):
    def __init__(self, texts, summaries, tokenizer, max_len_input, max_len_output):
        self.texts = texts
        self.summaries = summaries
        self.tokenizer = tokenizer
        self.max_len_input = max_len_input
        self.max_len_output = max_len_output

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        summary = self.summaries[idx]
        input_encoding = self.tokenizer.encode_plus(
            text,
            max_length=self.max_len_input,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        target_encoding = self.tokenizer.encode_plus(
            summary,
            max_length=self.max_len_output,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': input_encoding['input_ids'].flatten(),
            'attention_mask': input_encoding['attention_mask'].flatten(),
            'labels': target_encoding['input_ids'].flatten()
        }

tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')

MAX_LEN_INPUT = 50
MAX_LEN_OUTPUT = 20
BATCH_SIZE = 2
EPOCHS = 3
LEARNING_RATE = 1e-4

train_dataset = TextSummaryDataset(train_texts, train_summaries, tokenizer, MAX_LEN_INPUT, MAX_LEN_OUTPUT)
val_dataset = TextSummaryDataset(val_texts, val_summaries, tokenizer, MAX_LEN_INPUT, MAX_LEN_OUTPUT)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(EPOCHS):
    model.train()
    total_loss_train = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss_train += loss.item()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch + 1}, Train loss: {total_loss_train / len(train_loader)}')

    model.eval()
    total_loss_val = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss_val += loss.item()
    print(f'Epoch {epoch + 1}, Validation loss: {total_loss_val / len(val_loader)}')

print('Training complete.')

# Example usage
def summarize(text, model, tokenizer, max_len_input=50, max_len_output=20):
    model.eval()
    input_encoding = tokenizer.encode_plus(
        text,
        max_length=max_len_input,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    input_ids = input_encoding['input_ids'].to(device)
    attention_mask = input_encoding['attention_mask'].to(device)
    generated_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=max_len_output,
        num_beams=2,
        repetition_penalty=2.5,
        length_penalty=1.0,
        early_stopping=True
    )
    preds = [
        tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids
    ]
    return preds[0]

sample_text = "Natural language processing enables computers to understand and process human languages."
summary = summarize(sample_text, model, tokenizer)
print(f'Text: {sample_text}')
print(f'Summary: {summary}')


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Epoch 1, Train loss: 8.905749320983887
Epoch 1, Validation loss: 7.461780548095703
Epoch 2, Train loss: 7.765360116958618
Epoch 2, Validation loss: 6.153433799743652
Epoch 3, Train loss: 5.5194902420043945
Epoch 3, Validation loss: 4.8472900390625
Training complete.
Text: Natural language processing enables computers to understand and process human languages.
Summary: Die Verarbeitung von Natursprache ermöglicht es Computer, die Sprache zu verstehen und zu verarbeitet.


LDA

In [8]:
pip install gensim nltk


In [9]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim import corpora
from gensim.models import LdaModel
import pandas as pd

nltk.download('punkt')
nltk.download('stopwords')

data = {
    'text': [
        'Natural language processing is a field of artificial intelligence.',
        'Machine learning provides systems the ability to automatically learn.',
        'Deep learning is a subset of machine learning.',
        'Natural language processing involves the interaction between computers and humans.',
        'Artificial intelligence is intelligence demonstrated by machines.',
        'Machine learning algorithms build a model based on sample data.',
        'Deep learning models are inspired by the structure and function of the brain.',
        'Natural language processing includes speech recognition, text generation, and translation.',
        'Artificial intelligence is used in various applications such as robotics and autonomous driving.',
        'Machine learning and deep learning are important aspects of AI.'
    ]
}

df = pd.DataFrame(data)


stop_words = set(stopwords.words('english'))

def preprocess(text):
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return tokens

processed_docs = df['text'].apply(preprocess)

dictionary = corpora.Dictionary(processed_docs)
corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

num_topics = 3
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

for idx, topic in lda_model.print_topics(num_words=5):
    print(f'Topic {idx+1}: {topic}')

def infer_topic(text, lda_model, dictionary):
    processed_text = preprocess(text)
    bow = dictionary.doc2bow(processed_text)
    topics = lda_model.get_document_topics(bow)
    return topics

new_text = "Artificial intelligence and deep learning are transforming technology."
inferred_topics = infer_topic(new_text, lda_model, dictionary)
print(f'Inferred topics for the new text: {inferred_topics}')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Topic 1: 0.043*"artificial" + 0.042*"intelligence" + 0.042*"machine" + 0.042*"sample" + 0.042*"build"
Topic 2: 0.074*"natural" + 0.074*"processing" + 0.074*"language" + 0.031*"learning" + 0.031*"machine"
Topic 3: 0.134*"learning" + 0.083*"deep" + 0.060*"intelligence" + 0.058*"machine" + 0.034*"structure"
Inferred topics for the new text: [(0, 0.07309116), (1, 0.07060922), (2, 0.85629964)]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
